In [1]:
import sys
sys.path.append('/home/kaustubh/Documents/GitHub/Self_Driving_Car/')

In [2]:
from stable_baselines3 import SAC
from carla_gym_env import CarlaEnv
from custom_feature_extractor import CustomFeatureExtractor

['', '/opt/ros/humble/lib/python3.10/site-packages', '/opt/ros/humble/local/lib/python3.10/dist-packages', '/home/kaustubh/miniconda3/envs/carla_15/lib/python310.zip', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/lib-dynload', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/', '/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/setuptools/_vendor', '/tmp/tmpzb1p4uex', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model']


/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
def main():
    env = CarlaEnv()
    policy_kwargs = {
        'features_extractor_class' : CustomFeatureExtractor,
        'features_extractor_kwargs' : {},
        'net_arch' : dict(pi=[768, 256, 256], qf=[768, 256, 256])
    }
    
    model = SAC(
        "CnnPolicy",
        env,
        policy_kwargs=policy_kwargs,
        learning_rate=3e-4,
        buffer_size=10000,
        learning_starts=5000,
        batch_size=64,
        tau=0.005,
        gamma=0.99,
        train_freq=1,
        gradient_steps=1,
        verbose=1,
        device='cpu'
    )
    
    model.learn(total_timesteps=1000000)
    model.save("sac_carla_model")

In [ ]:
if __name__ == "__main__":
    main()

/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 23.04GB > 22.95GB
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


Lane Invaded
